# Find Diverse set based on Embeddings 

In [89]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix
import pandas as pd    
%matplotlib inline

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Creating the input data

In [91]:
df = pd.read_csv(r'/content/drive/MyDrive/SemSeg2/data/leftImg8bit_embed_EXP_3.csv')  # SAME EMBEDDING ALL SAMPLE SLICES IN EXP 1 
df = df.iloc[: , 1:]
df.head(10)

,RawImage,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.050060,0.186609,0.285789,-0.341629,0.299555,0.164631,-0.054196,0.003887,-0.088560,-0.028240,-0.099284,-0.212423,0.096199,-0.280873,-0.000521,-0.048372,0.170226,0.091123,-0.038271,0.251869,0.243154,-0.065795,-0.135999,0.147292,-0.192736,0.106965,0.140913,-0.191942,-0.017900,-0.295618,0.208899,-0.218925
1,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.060355,0.189354,0.277606,-0.340768,0.297376,0.158460,-0.078991,-0.016574,-0.082331,-0.014868,-0.108495,-0.236172,0.099194,-0.287159,-0.008142,-0.045915,0.154705,0.094528,-0.035247,0.239563,0.225566,-0.066278,-0.134329,0.124250,-0.186539,0.114072,0.141409,-0.184509,-0.013016,-0.317482,0.224002,-0.212105
2,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.059808,0.191247,0.275087,-0.340758,0.300066,0.158644,-0.079930,-0.019438,-0.085153,-0.025034,-0.110100,-0.233172,0.100532,-0.286715,-0.006623,-0.042730,0.157116,0.093205,-0.035197,0.235038,0.227301,-0.063859,-0.132561,0.129304,-0.187537,0.110020,0.145775,-0.183302,-0.010622,-0.312889,0.224558,-0.216981
3,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.033130,0.146860,0.284868,-0.315965,0.330242,0.136627,-0.088798,0.000765,-0.105067,-0.023773,-0.096416,-0.214994,0.086237,-0.245752,-0.030217,-0.023679,0.176444,0.122303,-0.040759,0.308981,0.238431,-0.064972,-0.173447,0.182539,-0.179113,0.087972,0.127956,-0.190022,0.020232,-0.309573,0.190299,-0.178478
4,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.056979,0.185239,0.283968,-0.341270,0.301531,0.157135,-0.070099,-0.010220,-0.085391,-0.019608,-0.105557,-0.224120,0.093418,-0.279347,-0.005940,-0.044242,0.162001,0.099380,-0.037410,0.251529,0.232169,-0.066996,-0.138462,0.136198,-0.187868,0.111220,0.140605,-0.188812,-0.009089,-0.310372,0.213960,-0.211490
5,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.017169,0.125371,0.279693,-0.301518,0.358755,0.110421,-0.108200,-0.008304,-0.114119,-0.021836,-0.105178,-0.218850,0.077335,-0.221262,-0.047661,-0.004754,0.175800,0.148041,-0.047997,0.316053,0.229918,-0.061953,-0.181580,0.206132,-0.170730,0.076950,0.119029,-0.189933,0.049855,-0.316335,0.179264,-0.159071
6,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.053756,-0.044214,0.220110,-0.142948,0.413729,-0.033845,-0.201580,-0.028967,-0.139357,-0.021194,-0.086410,-0.180473,0.001490,-0.028049,-0.164226,0.102429,0.151470,0.244595,-0.065097,0.461631,0.140818,-0.041474,-0.264264,0.294493,-0.083720,-0.008717,0.025171,-0.116841,0.171054,-0.287649,0.032785,0.040700
7,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.005206,0.068612,0.262982,-0.257944,0.396215,0.069414,-0.148861,-0.021141,-0.129340,-0.017641,-0.108610,-0.219968,0.066419,-0.164413,-0.085437,0.033635,0.168497,0.185487,-0.053574,0.372499,0.212712,-0.057507,-0.208663,0.239386,-0.143009,0.047396,0.084835,-0.177495,0.094181,-0.318593,0.145222,-0.106681
8,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,0.054621,-0.064989,0.214148,-0.125204,0.407493,-0.047567,-0.204688,-0.028412,-0.142068,-0.022288,-0.079122,-0.175456,-0.007450,-0.011129,-0.172466,0.108396,0.148482,0.246724,-0.061744,0.473361,0.131387,-0.042692,-0.270515,0.297782,-0.074304,-0.016351,0.016402,-0.107201,0.178626,-0.276248,0.014320,0.055914
9,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.076271,0.212738,0.276456,-0.360203,0.261767,0.181361,-0.049768,-0.010614,-0.072045,-0.018703,-0.103717,-0.225920,0.105453,-0.310019,0.016624,-0.067534,0.151320,0.065920,-0.028490,0.202267,0.226257,-0.067173,-0.110410,0.093576,-0.193842,0.127327,0.154960,-0.181440,-0.046478,-0.300294,0.231616,-0.238786


In [92]:
input_features=df.iloc[: , 1:]
input_features.shape

(2975, 32)

# SET SAMPLE SIZE

In [93]:
#Select the total clusters , also the number of samples we want at the end 
S=500    #also change depending on sample slice   !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# CLUSTERING 

In [94]:
#Set Up data sets to be referencing the same random set Cx and Number of randomly chosen clusers S as is applied to random. 
Cx=input_features
#total number clusters is the number of samples we want.
n_clusters=S

#Use best Cluster method that searches sample space , and finds groupings(clusters) that are similar
#The Agglomerative method will stop when the sample size has been reached
#Euclidian distance is used to find most similar groupings and put into the same cluster
#Ward linkage method is used to account of extremities
#Each point in random set is eventually given a cluster , and total number clusters is the number of samples we want.  
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')  
k=cluster.fit_predict(Cx)
Data=pd.DataFrame(Cx)
Cluster=pd.DataFrame(k , columns=['Cluster'])
result = pd.concat([Data, Cluster], axis=1)

#Select Nearest Center

In [95]:
#Many records per cluster , need to find the best distanced points 
#result['Cluster'].value_counts().sort_index(ascending=False).plot(kind="barh" , figsize=(30, 22))

In [96]:
#Find centroid of each cluster 
CenterMatrix=result.groupby("Cluster").agg({'0' : 'mean'  , '1' : 'mean', '2' : 'mean' , '3' : 'mean', '4' : 'mean' , '5' : 'mean', '6' : 'mean' , '7' : 'mean', '8' : 'mean' ,'9' : 'mean', '10' : 'mean' , '11' : 'mean', '12' : 'mean' , '13' : 'mean', '14' : 'mean' , '15' : 'mean', '16' : 'mean' ,'17' : 'mean', '18' : 'mean' , '19' : 'mean', '20' : 'mean' , '21' : 'mean', '22' : 'mean' , '23' : 'mean', '24' : 'mean' ,'25' : 'mean', '26' : 'mean' , '27' : 'mean', '28' : 'mean' , '29' : 'mean', '30' : 'mean' , '31' : 'mean'        })


In [97]:
#Make a center cluster for each of the image rows
H=pd.merge(result, CenterMatrix, how='inner', left_on='Cluster', right_on='Cluster')

In [98]:
#Compute each points distance from its center 


#importing numpy
import numpy as np

#initializing two arrays
array1 = np.array(H.iloc[ : ,33:])
array2 = np.array(result.iloc[0 , :-1 ])

#computing the Euclidan distance 
temp = array1 - array2
distance = np.sqrt(np.sum(np.square(temp)))


print("Euclidean Distance: ", distance)


Euclidean Distance:  35.08035783715016


In [99]:
#initializing two arrays
ls=list()
for i in range(len(H)):
  array1 = np.array(H.iloc[ i:i+1 ,33:]) #center points
  array2 = np.array(result.iloc[i , :-1 ]) #Matrix points
  temp = array1 - array2
  distance = np.sqrt(np.sum(np.square(temp)))
  ls.append([i,distance])

euclid = pd.DataFrame(ls,columns=["rec","distance"])

In [100]:
final = pd.concat([result, euclid], axis=1)


In [101]:
# Select for sample the true point that gives us the minimum distance 
out2=final.loc[final.groupby('Cluster').distance.idxmin()] #true points 

In [102]:
out2.shape

(500, 35)

In [103]:
out2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,Cluster,rec,distance
2061,0.020155,0.054386,0.266383,-0.239242,0.395844,0.047146,-0.146062,-0.017527,-0.128955,-0.021439,-0.093758,-0.206249,0.044683,-0.144827,-0.104948,0.040313,0.175423,0.199611,-0.059436,0.396862,0.199605,-0.060223,-0.234198,0.261337,-0.139504,0.041748,0.079692,-0.170373,0.105201,-0.314742,0.116729,-0.072474,0,2061,0.064740
2481,0.091478,-0.158131,0.153048,-0.009845,0.368101,-0.143228,-0.206248,-0.042391,-0.131433,-0.010401,-0.036804,-0.111653,-0.062332,0.100909,-0.215812,0.143205,0.101523,0.263833,-0.063799,0.473334,0.058658,-0.034658,-0.299683,0.299327,-0.014311,-0.049268,-0.049437,-0.050144,0.209633,-0.213331,-0.093258,0.180502,1,2481,0.108918
2136,0.086682,-0.142218,0.165713,-0.025570,0.370810,-0.125064,-0.204110,-0.029987,-0.130699,-0.015480,-0.039294,-0.114546,-0.062020,0.083584,-0.208144,0.141230,0.114169,0.260626,-0.062266,0.488159,0.064729,-0.031297,-0.297049,0.306301,-0.021654,-0.051630,-0.035870,-0.053233,0.205614,-0.220461,-0.081021,0.165202,2,2136,0.064840
2452,0.082274,-0.129578,0.169904,-0.037489,0.385513,-0.119716,-0.209474,-0.041145,-0.134752,-0.010129,-0.051309,-0.130274,-0.047325,0.068863,-0.205741,0.133323,0.114226,0.264205,-0.064812,0.478994,0.078171,-0.035231,-0.292123,0.300978,-0.029555,-0.041728,-0.030242,-0.062251,0.208759,-0.234110,-0.061748,0.152724,3,2452,0.048776
1615,-0.024647,0.124132,0.288074,-0.305156,0.356196,0.104584,-0.106186,-0.015261,-0.110662,-0.013739,-0.102873,-0.220661,0.071016,-0.215534,-0.049982,-0.005138,0.169408,0.151192,-0.051161,0.331655,0.220517,-0.069142,-0.190359,0.198653,-0.169098,0.083772,0.110410,-0.188212,0.047083,-0.321978,0.171769,-0.144663,4,1615,0.131511
374,0.002224,0.078853,0.268159,-0.271174,0.387184,0.072110,-0.150148,-0.029517,-0.117619,-0.018315,-0.107934,-0.223742,0.067402,-0.174905,-0.084197,0.025745,0.165893,0.181881,-0.050445,0.364009,0.206350,-0.063514,-0.208744,0.238949,-0.149422,0.057285,0.095178,-0.172329,0.086401,-0.320797,0.150629,-0.103330,5,374,0.050000
1607,-0.036475,0.156625,0.273847,-0.324025,0.336600,0.137713,-0.096353,-0.015744,-0.104027,-0.023733,-0.117873,-0.230951,0.097754,-0.253668,-0.026835,-0.021153,0.171934,0.120618,-0.040518,0.271965,0.230128,-0.066712,-0.148953,0.169996,-0.179410,0.091501,0.132555,-0.188411,0.019672,-0.310728,0.204149,-0.195481,6,1607,0.044881
1589,0.051177,-0.021019,0.224407,-0.166834,0.419720,-0.031154,-0.195389,-0.050326,-0.128862,-0.011722,-0.094261,-0.194124,0.018455,-0.061642,-0.155496,0.089017,0.146784,0.243398,-0.068862,0.440545,0.149061,-0.052103,-0.262983,0.280196,-0.096824,0.008706,0.033713,-0.128311,0.162387,-0.303423,0.055512,0.023788,7,1589,0.120639
2001,0.052037,-0.076406,0.221324,-0.117784,0.399435,-0.065727,-0.200034,-0.034742,-0.138913,-0.013137,-0.063094,-0.164864,-0.021673,-0.007463,-0.178575,0.100576,0.137510,0.249141,-0.058057,0.478627,0.122616,-0.053835,-0.289378,0.292173,-0.069881,-0.011421,0.006634,-0.093830,0.175490,-0.280643,-0.010548,0.079654,8,2001,0.050212
1909,0.055608,-0.039865,0.220879,-0.151225,0.416046,-0.036915,-0.189466,-0.035614,-0.136266,-0.015080,-0.087799,-0.180452,0.006041,-0.042672,-0.158757,0.100164,0.151145,0.242074,-0.070657,0.457242,0.146745,-0.047384,-0.264106,0.293477,-0.088948,-0.000297,0.024356,-0.127288,0.169287,-0.290784,0.038607,0.036052,9,1909,0.155165


In [104]:
#Link it to name of files on index 

selected=pd.merge(df, out2, left_index=True, right_index=True)
selected

,RawImage,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,27_x,28_x,29_x,30_x,31_x,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,24_y,25_y,26_y,27_y,28_y,29_y,30_y,31_y,Cluster,rec,distance
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.050060,0.186609,0.285789,-0.341629,0.299555,0.164631,-0.054196,0.003887,-0.088560,-0.028240,-0.099284,-0.212423,0.096199,-0.280873,-0.000521,-0.048372,0.170226,0.091123,-0.038271,0.251869,0.243154,-0.065795,-0.135999,0.147292,-0.192736,0.106965,0.140913,-0.191942,-0.017900,-0.295618,0.208899,-0.218925,-0.050060,0.186609,0.285789,-0.341629,0.299555,0.164631,-0.054196,0.003887,-0.088560,-0.028240,-0.099284,-0.212423,0.096199,-0.280873,-0.000521,-0.048372,0.170226,0.091123,-0.038271,0.251869,0.243154,-0.065795,-0.135999,0.147292,-0.192736,0.106965,0.140913,-0.191942,-0.017900,-0.295618,0.208899,-0.218925,61,0,0.015800
2,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.059808,0.191247,0.275087,-0.340758,0.300066,0.158644,-0.079930,-0.019438,-0.085153,-0.025034,-0.110100,-0.233172,0.100532,-0.286715,-0.006623,-0.042730,0.157116,0.093205,-0.035197,0.235038,0.227301,-0.063859,-0.132561,0.129304,-0.187537,0.110020,0.145775,-0.183302,-0.010622,-0.312889,0.224558,-0.216981,-0.059808,0.191247,0.275087,-0.340758,0.300066,0.158644,-0.079930,-0.019438,-0.085153,-0.025034,-0.110100,-0.233172,0.100532,-0.286715,-0.006623,-0.042730,0.157116,0.093205,-0.035197,0.235038,0.227301,-0.063859,-0.132561,0.129304,-0.187537,0.110020,0.145775,-0.183302,-0.010622,-0.312889,0.224558,-0.216981,200,2,0.051335
12,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.047509,0.174882,0.274067,-0.336136,0.317107,0.154719,-0.089318,-0.018256,-0.092248,-0.019835,-0.114631,-0.232220,0.102413,-0.276992,-0.014950,-0.040028,0.160322,0.103964,-0.037891,0.242850,0.233383,-0.065555,-0.135165,0.143447,-0.185328,0.102625,0.139976,-0.188612,0.001260,-0.307795,0.219165,-0.209501,-0.047509,0.174882,0.274067,-0.336136,0.317107,0.154719,-0.089318,-0.018256,-0.092248,-0.019835,-0.114631,-0.232220,0.102413,-0.276992,-0.014950,-0.040028,0.160322,0.103964,-0.037891,0.242850,0.233383,-0.065555,-0.135165,0.143447,-0.185328,0.102625,0.139976,-0.188612,0.001260,-0.307795,0.219165,-0.209501,320,12,0.052554
19,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.062186,0.193339,0.271927,-0.349418,0.293930,0.170187,-0.070057,-0.014857,-0.079100,-0.022329,-0.111371,-0.226859,0.105902,-0.291054,0.000508,-0.051417,0.155555,0.085798,-0.035345,0.220582,0.233752,-0.066649,-0.121353,0.122420,-0.188723,0.115231,0.146565,-0.185185,-0.020610,-0.305104,0.229324,-0.228038,-0.062186,0.193339,0.271927,-0.349418,0.293930,0.170187,-0.070057,-0.014857,-0.079100,-0.022329,-0.111371,-0.226859,0.105902,-0.291054,0.000508,-0.051417,0.155555,0.085798,-0.035345,0.220582,0.233752,-0.066649,-0.121353,0.122420,-0.188723,0.115231,0.146565,-0.185185,-0.020610,-0.305104,0.229324,-0.228038,18,19,0.032064
23,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.063242,0.194202,0.275126,-0.347201,0.292522,0.169150,-0.071129,-0.016636,-0.079609,-0.018981,-0.113259,-0.231173,0.108288,-0.294697,0.002159,-0.052682,0.153989,0.086521,-0.035118,0.225552,0.229344,-0.066450,-0.122584,0.115399,-0.186495,0.114211,0.144700,-0.185869,-0.023413,-0.306472,0.227363,-0.223757,-0.063242,0.194202,0.275126,-0.347201,0.292522,0.169150,-0.071129,-0.016636,-0.079609,-0.018981,-0.113259,-0.231173,0.108288,-0.294697,0.002159,-0.052682,0.153989,0.086521,-0.035118,0.225552,0.229344,-0.066450,-0.122584,0.115399,-0.186495,0.114211,0.144700,-0.185869,-0.023413,-0.306472,0.227363,-0.223757,76,23,0.027425
27,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,-0.059300,0.188632,0.277722,-0.344811,0.296084,0.162678,-0.077065,-0.015805,-0.082046,-0.016590,-0.110087,-0.232802,0.102754,-0.289018,-0.004059,-0.048890,0.156605,0.091098,-0.033869,0.237886,0.229200,-0.065861,-0.13

#Record Metric

In [105]:
#Total Distance and average distance for this set 

print(out2['distance'].sum())
print(out2['distance'].sum()/S)


78.09246619581427
0.15618493239162853


N100: 6.067435424053481
0.06067435424053481

N200: 
14.656578233587044
0.07328289116793522

N500: 78.09246619581427
0.15618493239162853

N1000 ; 235.2921074580062
0.2352921074580062

N1500 : 447.6879716378508
0.29845864775856723

N2000 : 673.6710897311274
        0.33683554486556366

N2500 : 861.9328205197057
0.3447731282078823

# CREATE FINAL DATAFRAME 

In [106]:
#fetch masks file names 
K=pd.DataFrame(selected['RawImage'])
K


,RawImage
0,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
2,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
12,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
19,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
23,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
27,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
28,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
29,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
30,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...
31,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...


In [107]:
#Fetch Corrisponding Ground Mask From Meta Data 
import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes")

m= pd.read_csv(r'CityScapes_MetaData.csv')
m.head(10)

,Unnamed: 0,ImageName,RawImage,Mask,ColourMask
0,0,aachen_000000_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
1,1,aachen_000001_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
2,2,aachen_000002_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
3,3,aachen_000003_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
4,4,aachen_000004_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
5,5,aachen_000005_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
6,6,aachen_000006_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
7,7,aachen_000007_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
8,8,aachen_000008_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
9,9,aachen_000009_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...


In [108]:
J = pd.merge(K, m, how="inner", on="RawImage")
J[['ImageName' ,	'RawImage','Mask'	, 'ColourMask']]

,ImageName,RawImage,Mask,ColourMask
0,aachen_000000_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
1,aachen_000002_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
2,aachen_000012_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
3,aachen_000019_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
4,aachen_000023_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
5,aachen_000027_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
6,aachen_000028_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
7,aachen_000029_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
8,aachen_000030_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...
9,aachen_000031_000019,/content/drive/MyDrive/SemSeg2/data/leftImg8bi...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...,/content/drive/MyDrive/SemSeg2/data/gtFine/tra...


#CHANGE FILE NAME

In [109]:
#Write to a data frame 

import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes/CoreSet_1")


#J.to_csv('CityScapes_Diversity_EXP3_100.csv')
#J.to_csv('CityScapes_Diversity_EXP3_200.csv')
J.to_csv('CityScapes_Diversity_EXP3_500.csv')
#J.to_csv('CityScapes_Diversity_EXP3_1000.csv')
#J.to_csv('CityScapes_Diversity_EXP3_1500.csv')
#J.to_csv('CityScapes_Diversity_EXP3_2000.csv')
#J.to_csv('CityScapes_Diversity_EXP3_2500.csv')

#SHOW VISUALS OF CLUSTERS 

In [79]:
pd.set_option('display.max_rows', None)
k=result[['Cluster']].value_counts()
k

Cluster
18         43
5          42
15         42
102        38
65         37
46         36
0          36
37         33
96         32
32         29
28         28
36         28
57         28
67         28
115        28
76         27
147        27
138        27
152        25
127        25
16         25
71         25
27         25
155        25
19         24
112        24
124        24
129        24
43         24
101        23
2          23
160        23
128        23
13         23
142        23
99         23
171        23
20         22
31         22
26         22
53         22
80         22
22         21
1          21
141        21
47         19
90         19
120        19
48         19
114        19
73         19
107        19
70         18
58         18
113        18
103        18
88         18
8          18
59         18
121        17
21         17
174        17
189        17
41         16
9          16
51         16
186        16
62         16
14         16
111        16
108        1

In [57]:
selectedX=pd.merge(df, result, left_index=True, right_index=True)
selectedX

Output hidden; open in https://colab.research.google.com to view.

In [58]:
view = selectedX[['RawImage','Cluster']]
v=view.loc[view['Cluster']==317]['RawImage'].tolist()
v

[]

In [59]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# loads the image in color
img = cv2.imread(v[0])
cv2_imshow(img)

IndexError: ignored

In [ ]:
img = cv2.imread(v[1])
cv2_imshow(img)

In [ ]:
img = cv2.imread(v[2])
cv2_imshow(img)

# POPULATE INTO FOLDERS ---> Clear and add to folder 

MAKE SURE ALLIGNMENT BETWEEN THE IMAGES AND MASKS 

In [110]:
#READ EXPERIMENT FILE 
import pandas as pd
import os
os.chdir("/content/drive/MyDrive/SemSeg2/Experiments/CityScapes/CoreSet_1")

#*****select one of the below********

#a Run from above 
df=J

#b run from stored selected file names list 
#Same core embedding different clustering seed
#df = pd.read_csv('CityScapes_Diversity_EXP1_100.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_200.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_500.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_1000.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_1500.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP1_2000.csv')  
#df = pd.read_csv('CityScapes_Diversity_EXP3_2500.csv')  



r = df['RawImage']
ls_mask = df['Mask']


In [111]:
len(df)

500

In [112]:
#clear the images
mydir = '/content/drive/MyDrive/SemSeg2/data_random/leftImg8bit/train/all'

filelist = [ f for f in os.listdir(mydir) ]
for f in filelist:
  os.remove(os.path.join(mydir, f))

#clear masks 
mydir = '/content/drive/MyDrive/SemSeg2/data_random/gtFine/train/all'

filelist = [ f for f in os.listdir(mydir) ]
for f in filelist:
  os.remove(os.path.join(mydir, f))

In [113]:
#Now copy into a directory into the two respoective folders 
#print(r)

import shutil

#clear all files but not the folder stucture 

for i in range(len(r)):
  shutil.copy(r[i], r'/content/drive/MyDrive/SemSeg2/data_random/leftImg8bit/train/all')
  print(r[i])

/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000002_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000012_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000019_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000023_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000027_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000028_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000029_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000030_000019_leftImg8bit.png
/content/drive/MyDrive/SemSeg2/data/leftImg8bit/train/aachen/aachen_000031_000019_leftImg8bit.png
/content/drive/MyDri

In [114]:
#Now copy into a directory into the two respective folders 
import shutil

#clear all files but not the folder stucture 

for i in range(len(ls_mask)):
  shutil.copy(ls_mask[i], r'/content/drive/MyDrive/SemSeg2/data_random/gtFine/train/all')
  print(ls_mask[i])

/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000000_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000002_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000012_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000019_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000023_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000027_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000028_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000029_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000030_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2/data/gtFine/train/aachen/aachen_000031_000019_gtFine_labelIds.png
/content/drive/MyDrive/SemSeg2